In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np

from triplet_loss import batch_hard_triplet_loss

# Loading and pre-processing the MNIST dataset

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 2s 0us/step


In [0]:
x_train = np.expand_dims(x_train,-1)
x_test = np.expand_dims(x_test,-1)

In [0]:
assert(x_train.shape[1:] == (28,28,1))
assert(x_test.shape[1:] == (28,28,1))

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [0]:
assert(y_train.dtype == np.uint8)
assert(y_test.dtype == np.uint8)

# Building a simple Convolutional Neural Network with Triplet Loss

In [0]:
batch_size = 64
epochs = 20
learning_rate = 1e-4
embedding_size = 64

In [0]:
# default in Tensorflow
K.set_image_data_format("channels_last")

In [0]:
def keras_batch_hard_triplet_loss(labels, y_pred):
  # As omoindrot's loss functions expect the labels to have shape (batch_size,), labels are flattaned.
  # Before flattening, they have shape (batch_size,1)
  
  labels = K.flatten(labels)
  return batch_hard_triplet_loss(labels, y_pred, margin = 0.5)

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

# last layer is not activated.
model.add(Dense(embedding_size, activation='linear'))
# Normalizing the created embeddings
model.add(Lambda(lambda  x: K.l2_normalize(x,axis=1)))


model.compile(loss=keras_batch_hard_triplet_loss,
              optimizer=keras.optimizers.Adam(learning_rate))

In [14]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

# Training the model

In [15]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 9s 158us/step - loss: 0.5390 - val_loss: 0.5005
Epoch 2/20
60000/60000 [==============================] - 9s 152us/step - loss: 0.5090 - val_loss: 0.4770
Epoch 3/20
60000/60000 [==============================] - 9s 151us/step - loss: 0.4137 - val_loss: 0.1964
Epoch 4/20
60000/60000 [==============================] - 9s 149us/step - loss: 0.2629 - val_loss: 0.1516
Epoch 5/20
60000/60000 [==============================] - 9s 151us/step - loss: 0.2124 - val_loss: 0.1299
Epoch 6/20
60000/60000 [==============================] - 9s 150us/step - loss: 0.1864 - val_loss: 0.1178
Epoch 7/20
60000/60000 [==============================] - 9s 152us/step - loss: 0.1670 - val_loss: 0.1094
Epoch 8/20
60000/60000 [==============================] - 9s 152us/step - loss: 0.1528 - val_loss: 0.1011
Epoch 9/20
60000/60000 [==============================] - 9s 151us/step - loss: 0.1391 - val_loss: 0.0

For further discussion on Triplet Loss / Triplet mining, please see omoindrot's work at https://omoindrot.github.io/triplet-loss and 
https://github.com/omoindrot/tensorflow-triplet-loss

The puspuse of this notebook is to show how to use omoindrot's triplet losses in Keras.